In [4]:
import os
import base64
import requests
import json
from pathlib import Path
import mimetypes
import datetime
import pandas as pd

# WordPress 설정
WP_URL = "https://prayer-church.co.kr"
WP_API = f"{WP_URL}/wp-json/wp/v2"
USERNAME = "prayerchurch"
APP_PASSWORD = "AziD L62i Ga4b 1efW ToAs hBMB"
FOLDER_PATH = "//Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/final"
REPORT_PATH = "//Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/report"

def get_image_files_recursive(folder_path):
    """지정된 폴더와 모든 하위 폴더에서 이미지 파일 가져오기"""
    이미지_확장자 = ['.jpg', '.jpeg', '.png', '.gif', '.webp']
    이미지_파일들 = []
    
    # os.walk를 사용하여 폴더와 모든 하위 폴더 탐색
    for 현재_경로, 하위_폴더들, 파일들 in os.walk(folder_path):
        for 파일 in 파일들:
            if any(파일.lower().endswith(확장자) for 확장자 in 이미지_확장자):
                이미지_파일들.append(os.path.join(현재_경로, 파일))
    
    return 이미지_파일들

def get_metadata_from_filename(filename):
    """파일명에서 메타데이터 추출"""
    # 경로와 확장자 없이 파일명만 가져오기
    기본_이름 = os.path.basename(filename)
    확장자_없는_이름 = os.path.splitext(기본_이름)[0]
    
    # 하이픈을 공백으로 대체
    이름_텍스트 = 확장자_없는_이름.replace('-', ' ')
    
    # 메타데이터 생성
    대체_텍스트 = 이름_텍스트
    캡션 = f"{이름_텍스트}"
    제목 = 이름_텍스트.title()  # 각 단어의 첫 글자 대문자화
    
    return {
        'alt_text': 대체_텍스트,
        'caption': 캡션,
        'title': 제목
    }

def create_schema_markup(metadata, image_url):
    """이미지용 schema.org 마크업 생성"""
    스키마 = {
        "@context": "https://schema.org/",
        "@type": "ImageObject",
        "contentUrl": image_url,
        "name": metadata['title'],
        "description": metadata['alt_text'],
        "caption": metadata['caption']
    }
    
    # ensure_ascii=False를 추가하여 한글이 올바르게 인코딩되도록 설정
    return json.dumps(스키마, ensure_ascii=False)

def upload_to_wordpress(image_path, metadata):
    """메타데이터와 함께 WordPress에 이미지 업로드"""
    try:
        # 인증 헤더 - Basic Auth
        인증 = requests.auth.HTTPBasicAuth(USERNAME, APP_PASSWORD)
        
        # 파일 이름 추출
        파일_이름 = os.path.basename(image_path)
        
        # 이미지 크기 정보 가져오기
        from PIL import Image
        with Image.open(image_path) as img:
            이미지_너비, 이미지_높이 = img.size
        
        # multipart/form-data로 파일 업로드
        with open(image_path, 'rb') as img:
            파일_데이터 = {'file': (파일_이름, img)}
            
            # 미디어 엔드포인트 POST 요청
            업로드_URL = f"{WP_API}/media"
            
            # 파일 업로드
            응답 = requests.post(
                업로드_URL,
                auth=인증,
                files=파일_데이터
            )
        
        if 응답.status_code in [200, 201]:
            응답_데이터 = 응답.json()
            이미지_URL = 응답_데이터.get('source_url', '')
            이미지_ID = 응답_데이터.get('id', '')
            
            # 업로드 성공 후 메타데이터 업데이트
            if 이미지_ID:
                # schema.org 마크업 생성 (이제 URL 있음)
                업데이트_스키마 = create_schema_markup(metadata, 이미지_URL)
                
                # 이미지 메타데이터 업데이트 - width와 height 추가
                업데이트_데이터 = {
                    'title': metadata['title'],
                    'alt_text': metadata['alt_text'],
                    'caption': metadata['caption'],
                    'description': 업데이트_스키마,
                    'meta': {
                        'width': 이미지_너비,
                        'height': 이미지_높이
                    }
                }
                
                # 메타데이터 업데이트 요청 (JSON 형식)
                헤더 = {
                    'Content-Type': 'application/json',
                    'Accept': 'application/json'
                }
                
                업데이트_URL = f"{WP_API}/media/{이미지_ID}"
                업데이트_응답 = requests.post(
                    업데이트_URL,
                    auth=인증,
                    headers=헤더,
                    json=업데이트_데이터
                )
                
                # 업데이트 결과 확인
                if 업데이트_응답.status_code not in [200, 201]:
                    print(f"메타데이터 업데이트 실패: {업데이트_응답.text}")
                
            return {
                'success': True,
                'id': 이미지_ID,
                'url': 이미지_URL,
                'width': 이미지_너비,
                'height': 이미지_높이,
                'title': metadata['title'],
                'alt_text': metadata['alt_text'],
                'caption': metadata['caption'],
                'response': 응답_데이터
            }
        else:
            return {
                'success': False,
                'status_code': 응답.status_code,
                'error': 응답.text
            }
    except Exception as e:
        return {
            'success': False,
            'error': str(e)
        }

def save_results_to_excel(results, folder_path):
    """결과를 엑셀 파일로 저장"""
    # 오늘 날짜 가져오기
    오늘 = datetime.datetime.now().strftime('%Y-%m-%d')
    
    # 결과 데이터 준비
    엑셀_데이터 = []
    
    for 항목 in results:
        파일명 = 항목['file_name']
        파일경로 = 항목['file_path']  # 전체 파일 경로 추가
        업로드_결과 = 항목['upload_result']
        
        if 업로드_결과['success']:
            행_데이터 = {
                '파일명': 파일명,
                '파일경로': 파일경로,  # 전체 파일 경로 추가
                '상태': '성공',
                'ID': 업로드_결과.get('id', ''),
                'URL': 업로드_결과.get('url', ''),
                '제목': 업로드_결과.get('title', ''),
                '대체텍스트': 업로드_결과.get('alt_text', ''),
                '캡션': 업로드_결과.get('caption', '')
            }
        else:
            행_데이터 = {
                '파일명': 파일명,
                '파일경로': 파일경로,  # 전체 파일 경로 추가
                '상태': '실패',
                '오류 코드': 업로드_결과.get('status_code', ''),
                '오류 메시지': 업로드_결과.get('error', '')
            }
        
        엑셀_데이터.append(행_데이터)
    
    # 데이터프레임 생성
    df = pd.DataFrame(엑셀_데이터)
    
    # 엑셀 파일 이름 생성
    파일명 = f"워드프레스_이미지_업로드_결과_{오늘}.xlsx"
    파일_경로 = os.path.join(folder_path, 파일명)
    
    # 지정된 폴더가 없으면 생성
    os.makedirs(folder_path, exist_ok=True)
    
    # 엑셀 파일로 저장
    df.to_excel(파일_경로, index=False, engine='openpyxl')
    
    return 파일_경로

def main():
    # 폴더와 모든 하위 폴더에서 이미지 파일 가져오기
    이미지_파일들 = get_image_files_recursive(FOLDER_PATH)
    
    # 이미지 개수 확인
    print(f"총 {len(이미지_파일들)}개의 이미지 파일을 찾았습니다.")
    
    결과 = []
    
    # 각 이미지 처리
    for 인덱스, 이미지_경로 in enumerate(이미지_파일들, 1):
        파일_이름 = os.path.basename(이미지_경로)
        상대_경로 = os.path.relpath(이미지_경로, FOLDER_PATH)
        
        print(f"[{인덱스}/{len(이미지_파일들)}] '{상대_경로}' 처리 중...")
        
        # 파일명에서 메타데이터 추출
        메타데이터 = get_metadata_from_filename(이미지_경로)
        
        # WordPress에 업로드
        업로드_결과 = upload_to_wordpress(이미지_경로, 메타데이터)
        
        # 결과 저장
        이미지_정보 = {
            'file_name': 파일_이름,
            'file_path': 상대_경로,  # 상대 경로 저장
            'upload_result': 업로드_결과
        }
        결과.append(이미지_정보)
        
        # 결과 출력
        if 업로드_결과['success']:
            print(f"✅ 성공: '{상대_경로}' 업로드 완료, URL: {업로드_결과['url']}")
        else:
            print(f"❌ 실패: '{상대_경로}' 업로드 실패, 오류: {업로드_결과.get('error', '알 수 없는 오류')}")
    
    # 모든 결과 요약
    성공_수 = sum(1 for 항목 in 결과 if 항목['upload_result']['success'])
    실패_수 = len(결과) - 성공_수
    
    print("\n===== 업로드 결과 요약 =====")
    print(f"총 이미지: {len(결과)}")
    print(f"성공: {성공_수}")
    print(f"실패: {실패_수}")
    
    # 모든 성공한 URL 출력
    if 성공_수 > 0:
        print("\n===== 업로드된 이미지 URL =====")
        for 항목 in 결과:
            if 항목['upload_result']['success']:
                print(f"{항목['file_path']}: {항목['upload_result']['url']}")
    
    # 결과를 엑셀 파일로 저장
    엑셀_파일_경로 = save_results_to_excel(결과, REPORT_PATH)
    print(f"\n✅ 결과가 엑셀 파일로 저장되었습니다: {엑셀_파일_경로}")

if __name__ == "__main__":
    main()

총 6개의 이미지 파일을 찾았습니다.
[1/6] '5 월 주일 대표 기도문/banner_5 월 주일 대표 기도문.png' 처리 중...
✅ 성공: '5 월 주일 대표 기도문/banner_5 월 주일 대표 기도문.png' 업로드 완료, URL: https://prayer-church.co.kr/wp-content/uploads/2025/05/banner_5-월-주일-대표-기도문.png
[2/6] '6월 셋째 주일 대표 기도문 모음 (2025년)/banner_6월 셋째 주일 대표 기도문 모음 (2025년).png' 처리 중...
✅ 성공: '6월 셋째 주일 대표 기도문 모음 (2025년)/banner_6월 셋째 주일 대표 기도문 모음 (2025년).png' 업로드 완료, URL: https://prayer-church.co.kr/wp-content/uploads/2025/05/banner_6월-셋째-주일-대표-기도문-모음-2025년.png
[3/6] '5 월 넷째 주 대표 기도문 모음/banner_5 월 넷째 주 대표 기도문 모음.png' 처리 중...
✅ 성공: '5 월 넷째 주 대표 기도문 모음/banner_5 월 넷째 주 대표 기도문 모음.png' 업로드 완료, URL: https://prayer-church.co.kr/wp-content/uploads/2025/05/banner_5-월-넷째-주-대표-기도문-모음.png
[4/6] '6월 둘째 주 대표 기도문 모음 (2025년)/banner_6월 둘째 주 대표 기도문 모음 (2025년).png' 처리 중...
✅ 성공: '6월 둘째 주 대표 기도문 모음 (2025년)/banner_6월 둘째 주 대표 기도문 모음 (2025년).png' 업로드 완료, URL: https://prayer-church.co.kr/wp-content/uploads/2025/05/banner_6월-둘째-주-대표-기도문-모음-2025년.png
[5/6] '5월 셋째 주일 대표 기도문/banner_5월 셋째 주일 대표 기도문.png' 처리

In [3]:
import os
import base64
import requests
import json
from pathlib import Path
import mimetypes
import datetime
import pandas as pd

# WordPress 설정
WP_URL = "https://prayer-church.co.kr"
WP_API = f"{WP_URL}/wp-json/wp/v2"
USERNAME = "prayerchurch"
APP_PASSWORD = "AziD L62i Ga4b 1efW ToAs hBMB"
FOLDER_PATH = "//Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/final"
REPORT_PATH = "//Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/report"

def get_image_files(folder_path):
    """폴더에서 모든 이미지 파일 가져오기"""
    이미지_확장자 = ['.jpg', '.jpeg', '.png', '.gif', '.webp']
    이미지_파일들 = []
    
    for 파일 in os.listdir(folder_path):
        if any(파일.lower().endswith(확장자) for 확장자 in 이미지_확장자):
            이미지_파일들.append(os.path.join(folder_path, 파일))
    
    return 이미지_파일들

def get_metadata_from_filename(filename):
    """파일명에서 메타데이터 추출"""
    # 경로와 확장자 없이 파일명만 가져오기
    기본_이름 = os.path.basename(filename)
    확장자_없는_이름 = os.path.splitext(기본_이름)[0]
    
    # 하이픈을 공백으로 대체
    이름_텍스트 = 확장자_없는_이름.replace('-', ' ')
    
    # 메타데이터 생성
    대체_텍스트 = 이름_텍스트
    캡션 = f"{이름_텍스트}"
    제목 = 이름_텍스트.title()  # 각 단어의 첫 글자 대문자화
    
    return {
        'alt_text': 대체_텍스트,
        'caption': 캡션,
        'title': 제목
    }

def create_schema_markup(metadata, image_url):
    """이미지용 schema.org 마크업 생성"""
    스키마 = {
        "@context": "https://schema.org/",
        "@type": "ImageObject",
        "contentUrl": image_url,
        "name": metadata['title'],
        "description": metadata['alt_text'],
        "caption": metadata['caption']
    }
    
    # ensure_ascii=False를 추가하여 한글이 올바르게 인코딩되도록 설정
    return json.dumps(스키마, ensure_ascii=False)

def upload_to_wordpress(image_path, metadata):
    """메타데이터와 함께 WordPress에 이미지 업로드"""
    # 인증 헤더 - Basic Auth
    인증 = requests.auth.HTTPBasicAuth(USERNAME, APP_PASSWORD)
    
    # 파일 이름 추출
    파일_이름 = os.path.basename(image_path)
    
    # 이미지 크기 정보 가져오기 - 업데이트된 부분
    from PIL import Image
    with Image.open(image_path) as img:
        이미지_너비, 이미지_높이 = img.size
    
    # multipart/form-data로 파일 업로드
    with open(image_path, 'rb') as img:
        파일_데이터 = {'file': (파일_이름, img)}
        
        # 미디어 엔드포인트 POST 요청
        업로드_URL = f"{WP_API}/media"
        
        # 파일 업로드
        응답 = requests.post(
            업로드_URL,
            auth=인증,
            files=파일_데이터
        )
    
    if 응답.status_code in [200, 201]:
        응답_데이터 = 응답.json()
        이미지_URL = 응답_데이터.get('source_url', '')
        이미지_ID = 응답_데이터.get('id', '')
        
        # 업로드 성공 후 메타데이터 업데이트
        if 이미지_ID:
            # schema.org 마크업 생성 (이제 URL 있음)
            업데이트_스키마 = create_schema_markup(metadata, 이미지_URL)
            
            # 이미지 메타데이터 업데이트 - width와 height 추가
            업데이트_데이터 = {
                'title': metadata['title'],
                'alt_text': metadata['alt_text'],
                'caption': metadata['caption'],
                'description': 업데이트_스키마,
                'meta': {
                    'width': 이미지_너비,
                    'height': 이미지_높이
                }
            }
            
            # 메타데이터 업데이트 요청 (JSON 형식)
            헤더 = {
                'Content-Type': 'application/json',
                'Accept': 'application/json'
            }
            
            업데이트_URL = f"{WP_API}/media/{이미지_ID}"
            업데이트_응답 = requests.post(
                업데이트_URL,
                auth=인증,
                headers=헤더,
                json=업데이트_데이터
            )
            
            # 업데이트 결과 확인
            if 업데이트_응답.status_code not in [200, 201]:
                print(f"메타데이터 업데이트 실패: {업데이트_응답.text}")
            
        return {
            'success': True,
            'id': 이미지_ID,
            'url': 이미지_URL,
            'width': 이미지_너비,   # 너비 정보 추가
            'height': 이미지_높이,  # 높이 정보 추가
            'title': metadata['title'],
            'alt_text': metadata['alt_text'],
            'caption': metadata['caption'],
            'response': 응답_데이터
        }

def save_results_to_excel(results, folder_path):
    """결과를 엑셀 파일로 저장"""
    # 오늘 날짜 가져오기
    오늘 = datetime.datetime.now().strftime('%Y-%m-%d')
    
    # 결과 데이터 준비
    엑셀_데이터 = []
    
    for 항목 in results:
        파일명 = 항목['file_name']
        업로드_결과 = 항목['upload_result']
        
        if 업로드_결과['success']:
            행_데이터 = {
                '파일명': 파일명,
                '상태': '성공',
                'ID': 업로드_결과.get('id', ''),
                'URL': 업로드_결과.get('url', ''),
                '제목': 업로드_결과.get('title', ''),
                '대체텍스트': 업로드_결과.get('alt_text', ''),
                '캡션': 업로드_결과.get('caption', '')
            }
        else:
            행_데이터 = {
                '파일명': 파일명,
                '상태': '실패',
                '오류 코드': 업로드_결과.get('status_code', ''),
                '오류 메시지': 업로드_결과.get('error', '')
            }
        
        엑셀_데이터.append(행_데이터)
    
    # 데이터프레임 생성
    df = pd.DataFrame(엑셀_데이터)
    
    # 엑셀 파일 이름 생성
    파일명 = f"워드프레스_이미지_업로드_결과_{오늘}.xlsx"
    파일_경로 = os.path.join(folder_path, 파일명)
    
    # 지정된 폴더가 없으면 생성
    os.makedirs(folder_path, exist_ok=True)
    
    # 엑셀 파일로 저장
    df.to_excel(파일_경로, index=False, engine='openpyxl')
    
    return 파일_경로

def main():
    # 폴더에서 모든 이미지 파일 가져오기
    이미지_파일들 = get_image_files(FOLDER_PATH)
    
    결과 = []
    
    # 각 이미지 처리
    for 이미지_경로 in 이미지_파일들:
        파일_이름 = os.path.basename(이미지_경로)
        print(f"'{파일_이름}' 처리 중...")
        
        # 파일명에서 메타데이터 추출
        메타데이터 = get_metadata_from_filename(이미지_경로)
        
        # WordPress에 업로드
        업로드_결과 = upload_to_wordpress(이미지_경로, 메타데이터)
        
        # 결과 저장
        이미지_정보 = {
            'file_name': 파일_이름,
            'upload_result': 업로드_결과
        }
        결과.append(이미지_정보)
        
        # 결과 출력
        if 업로드_결과['success']:
            print(f"✅ 성공: '{파일_이름}' 업로드 완료, URL: {업로드_결과['url']}")
        else:
            print(f"❌ 실패: '{파일_이름}' 업로드 실패, 오류: {업로드_결과.get('error', '알 수 없는 오류')}")
    
    # 모든 결과 요약
    성공_수 = sum(1 for 항목 in 결과 if 항목['upload_result']['success'])
    실패_수 = len(결과) - 성공_수
    
    print("\n===== 업로드 결과 요약 =====")
    print(f"총 이미지: {len(결과)}")
    print(f"성공: {성공_수}")
    print(f"실패: {실패_수}")
    
    # 모든 성공한 URL 출력
    if 성공_수 > 0:
        print("\n===== 업로드된 이미지 URL =====")
        for 항목 in 결과:
            if 항목['upload_result']['success']:
                print(f"{항목['file_name']}: {항목['upload_result']['url']}")
    
    # 결과를 엑셀 파일로 저장
    엑셀_파일_경로 = save_results_to_excel(결과, REPORT_PATH)
    print(f"\n✅ 결과가 엑셀 파일로 저장되었습니다: {엑셀_파일_경로}")

if __name__ == "__main__":
    main()


===== 업로드 결과 요약 =====
총 이미지: 0
성공: 0
실패: 0

✅ 결과가 엑셀 파일로 저장되었습니다: //Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/report/워드프레스_이미지_업로드_결과_2025-05-14.xlsx


In [1]:
import os
import base64
import requests
import json
from pathlib import Path
import mimetypes
import datetime
import pandas as pd

# WordPress 설정
WP_URL = "https://prayer-church.co.kr"
WP_API = f"{WP_URL}/wp-json/wp/v2"
USERNAME = "prayerchurch"
APP_PASSWORD = "AziD L62i Ga4b 1efW ToAs hBMB"
FOLDER_PATH = "//Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/final"
REPORT_PATH = "//Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/report"

def get_image_files(folder_path):
    """폴더에서 모든 이미지 파일 가져오기"""
    이미지_확장자 = ['.jpg', '.jpeg', '.png', '.gif', '.webp']
    이미지_파일들 = []
    
    for 파일 in os.listdir(folder_path):
        if any(파일.lower().endswith(확장자) for 확장자 in 이미지_확장자):
            이미지_파일들.append(os.path.join(folder_path, 파일))
    
    return 이미지_파일들

def get_metadata_from_filename(filename):
    """파일명에서 메타데이터 추출"""
    # 경로와 확장자 없이 파일명만 가져오기
    기본_이름 = os.path.basename(filename)
    확장자_없는_이름 = os.path.splitext(기본_이름)[0]
    
    # 하이픈을 공백으로 대체
    이름_텍스트 = 확장자_없는_이름.replace('-', ' ')
    
    # 메타데이터 생성
    대체_텍스트 = 이름_텍스트
    캡션 = f"{이름_텍스트}"
    제목 = 이름_텍스트.title()  # 각 단어의 첫 글자 대문자화
    
    return {
        'alt_text': 대체_텍스트,
        'caption': 캡션,
        'title': 제목
    }

def create_schema_markup(metadata, image_url):
    """이미지용 schema.org 마크업 생성"""
    스키마 = {
        "@context": "https://schema.org/",
        "@type": "ImageObject",
        "contentUrl": image_url,
        "name": metadata['title'],
        "description": metadata['alt_text'],
        "caption": metadata['caption']
    }
    
    # ensure_ascii=False를 추가하여 한글이 올바르게 인코딩되도록 설정
    return json.dumps(스키마, ensure_ascii=False)

def upload_to_wordpress(image_path, metadata):
    """메타데이터와 함께 WordPress에 이미지 업로드"""
    # 인증 헤더 - Basic Auth
    인증 = requests.auth.HTTPBasicAuth(USERNAME, APP_PASSWORD)
    
    # 파일 이름 추출
    파일_이름 = os.path.basename(image_path)
    
    # multipart/form-data로 파일 업로드
    with open(image_path, 'rb') as img:
        파일_데이터 = {'file': (파일_이름, img)}
        
        # 미디어 엔드포인트 POST 요청
        업로드_URL = f"{WP_API}/media"
        
        # 파일 업로드
        응답 = requests.post(
            업로드_URL,
            auth=인증,
            files=파일_데이터
        )
    
    if 응답.status_code in [200, 201]:
        응답_데이터 = 응답.json()
        이미지_URL = 응답_데이터.get('source_url', '')
        이미지_ID = 응답_데이터.get('id', '')
        
        # 업로드 성공 후 메타데이터 업데이트
        if 이미지_ID:
            # schema.org 마크업 생성 (이제 URL 있음)
            업데이트_스키마 = create_schema_markup(metadata, 이미지_URL)
            
            # 이미지 메타데이터 업데이트
            업데이트_데이터 = {
                'title': metadata['title'],
                'alt_text': metadata['alt_text'],
                'caption': metadata['caption'],
                'description': 업데이트_스키마
            }
            
            # 메타데이터 업데이트 요청 (JSON 형식)
            헤더 = {
                'Content-Type': 'application/json',
                'Accept': 'application/json'
            }
            
            업데이트_URL = f"{WP_API}/media/{이미지_ID}"
            업데이트_응답 = requests.post(
                업데이트_URL,
                auth=인증,
                headers=헤더,
                json=업데이트_데이터
            )
            
            # 업데이트 결과 확인
            if 업데이트_응답.status_code not in [200, 201]:
                print(f"메타데이터 업데이트 실패: {업데이트_응답.text}")
            
        return {
            'success': True,
            'id': 이미지_ID,
            'url': 이미지_URL,
            'title': metadata['title'],
            'alt_text': metadata['alt_text'],
            'caption': metadata['caption'],
            'response': 응답_데이터
        }
    else:
        return {
            'success': False,
            'status_code': 응답.status_code,
            'error': 응답.text
        }

def save_results_to_excel(results, folder_path):
    """결과를 엑셀 파일로 저장"""
    # 오늘 날짜 가져오기
    오늘 = datetime.datetime.now().strftime('%Y-%m-%d')
    
    # 결과 데이터 준비
    엑셀_데이터 = []
    
    for 항목 in results:
        파일명 = 항목['file_name']
        업로드_결과 = 항목['upload_result']
        
        if 업로드_결과['success']:
            행_데이터 = {
                '파일명': 파일명,
                '상태': '성공',
                'ID': 업로드_결과.get('id', ''),
                'URL': 업로드_결과.get('url', ''),
                '제목': 업로드_결과.get('title', ''),
                '대체텍스트': 업로드_결과.get('alt_text', ''),
                '캡션': 업로드_결과.get('caption', '')
            }
        else:
            행_데이터 = {
                '파일명': 파일명,
                '상태': '실패',
                '오류 코드': 업로드_결과.get('status_code', ''),
                '오류 메시지': 업로드_결과.get('error', '')
            }
        
        엑셀_데이터.append(행_데이터)
    
    # 데이터프레임 생성
    df = pd.DataFrame(엑셀_데이터)
    
    # 엑셀 파일 이름 생성
    파일명 = f"워드프레스_이미지_업로드_결과_{오늘}.xlsx"
    파일_경로 = os.path.join(folder_path, 파일명)
    
    # 지정된 폴더가 없으면 생성
    os.makedirs(folder_path, exist_ok=True)
    
    # 엑셀 파일로 저장
    df.to_excel(파일_경로, index=False, engine='openpyxl')
    
    return 파일_경로

def main():
    # 폴더에서 모든 이미지 파일 가져오기
    이미지_파일들 = get_image_files(FOLDER_PATH)
    
    결과 = []
    
    # 각 이미지 처리
    for 이미지_경로 in 이미지_파일들:
        파일_이름 = os.path.basename(이미지_경로)
        print(f"'{파일_이름}' 처리 중...")
        
        # 파일명에서 메타데이터 추출
        메타데이터 = get_metadata_from_filename(이미지_경로)
        
        # WordPress에 업로드
        업로드_결과 = upload_to_wordpress(이미지_경로, 메타데이터)
        
        # 결과 저장
        이미지_정보 = {
            'file_name': 파일_이름,
            'upload_result': 업로드_결과
        }
        결과.append(이미지_정보)
        
        # 결과 출력
        if 업로드_결과['success']:
            print(f"✅ 성공: '{파일_이름}' 업로드 완료, URL: {업로드_결과['url']}")
        else:
            print(f"❌ 실패: '{파일_이름}' 업로드 실패, 오류: {업로드_결과.get('error', '알 수 없는 오류')}")
    
    # 모든 결과 요약
    성공_수 = sum(1 for 항목 in 결과 if 항목['upload_result']['success'])
    실패_수 = len(결과) - 성공_수
    
    print("\n===== 업로드 결과 요약 =====")
    print(f"총 이미지: {len(결과)}")
    print(f"성공: {성공_수}")
    print(f"실패: {실패_수}")
    
    # 모든 성공한 URL 출력
    if 성공_수 > 0:
        print("\n===== 업로드된 이미지 URL =====")
        for 항목 in 결과:
            if 항목['upload_result']['success']:
                print(f"{항목['file_name']}: {항목['upload_result']['url']}")
    
    # 결과를 엑셀 파일로 저장
    엑셀_파일_경로 = save_results_to_excel(결과, REPORT_PATH)
    print(f"\n✅ 결과가 엑셀 파일로 저장되었습니다: {엑셀_파일_경로}")

if __name__ == "__main__":
    main()

'대표-기도문-나눔터-모음.jpg' 처리 중...
✅ 성공: '대표-기도문-나눔터-모음.jpg' 업로드 완료, URL: https://prayer-church.co.kr/wp-content/uploads/2025/05/대표-기도문-나눔터-모음.jpg
'대표-기도문-나눔터-모음.webp' 처리 중...
✅ 성공: '대표-기도문-나눔터-모음.webp' 업로드 완료, URL: https://prayer-church.co.kr/wp-content/uploads/2025/05/대표-기도문-나눔터-모음.webp

===== 업로드 결과 요약 =====
총 이미지: 2
성공: 2
실패: 0

===== 업로드된 이미지 URL =====
대표-기도문-나눔터-모음.jpg: https://prayer-church.co.kr/wp-content/uploads/2025/05/대표-기도문-나눔터-모음.jpg
대표-기도문-나눔터-모음.webp: https://prayer-church.co.kr/wp-content/uploads/2025/05/대표-기도문-나눔터-모음.webp

✅ 결과가 엑셀 파일로 저장되었습니다: //Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/report/워드프레스_이미지_업로드_결과_2025-05-08.xlsx
